# Mixing Station Workspace

***

Run the below code block to create a mixing_station instance, using the device names and com port addresses provided [here](data/devices/mixing_stations.json).

In [ ]:
%load_ext autoreload
%autoreload 2

from src.robot_controller import scheduler

instance = scheduler.experiment(device_name="microtron_02", home=False)

***

## Some Theory

The mixing station pulls information on each constituent electrolyte from a CSV file when creating a new electrolyte mixture. The constituents are numbered, which corresponds to the location of their container in the physical workspace. See the required information for each constituent below..

| # | Name | Dose Volume (uL) | Container Volume (mL) | Density (g/mL) | Aspirate Constant (mbar/uL) | Aspirate Speed (uL/s) |
| --- | --- | --- | --- | --- | --- | --- |

Based on the [literature](https://www.theleeco.com/uploads/2023/06/AN049-Pipetting-Disc-Pump-Application-Note-1.pdf), we can expect the aspirate constant to be roughly equal to the system pressure $P_r$ divided by the reservoir volume $V_r$. The system pressure can be assumed equal to atmospheric pressure $\approx$ 1000mbar and the chosen reservoir volume is 2500uL $\implies$ the constant should be about **0.4mbar/uL**.

$$ \Delta P_r = \frac{P_s}{V_r} V_{asp} $$ 

For more viscous liquids, this value may increase based on the ohm's law equivalent of fluid flow through a pipe (the Hagen-Poiseuille equation). To compensate for this, an extra variable is included to slow down the rate of aspiration in an attempt to lower the pressure change required to aspirate more viscous fluids. Along with the response time of the pressure PID controller, the aspirate speed will determine how quickly the pressure of the disc pump changes. A typical value would be **10uL/s**.

$$ \Delta P = \frac{8 \mu L}{\pi R^4} Q $$ 

***

## Tune Aspiration Variables

**The aspirate speed can be set to zero, to jump straight to the aspiration pressure, for low viscosity liquids.**

You can run the below code blocks to perform automatic tuning, where the machine will loop through the parameter ranges and use mass balance data to measure the errors. Tuning therefore requires the mass balance to be connected, and that the mixing chamber outputs to a container on the mass balance.

In [ ]:
%matplotlib inline
instance.tune(pot_number = 5, asp_const = [0.37, 0.39], aspirate_volume = [300.0, 2100.0], container_volume = 35, asp_speed = 0, density = 1.0, N = 3, M = 3)

**Aspiration tuning can also be run for a range of aspiration speeds..** 

In [ ]:
%matplotlib inline
for speed in [0, 100, 50, 20, 10]: #uL/s
    instance.tune(pot_number = 1, asp_const = [0.3, 0.8], aspirate_volume = [1.0, 50.0], container_volume = 35, asp_speed = speed, density = 1.0, N = 5, M = 5)

### Update [this](data/CSVs/electrolyte_recipe.csv) CSV file with the chosen variables before beginning an experiment! 

***

## Run Experiment

Running the below code block will begin a single experiment that creates an electrolye mixture based on the volumes of constitutent electrolytes given in the [electrolyte receipe csv file](data/recipes/electrolyte_recipe.csv).

In [ ]:
instance.csv_filename = "electrolyte_recipe.csv"
instance.read_csv()

instance.run()

***

### Run life test

Run *N* number of experiments in succession, using the parameters defined in the [life test csv file](data/recipes/life_test.csv).

In [ ]:
instance.csv_filename = "life_test.csv"
instance.read_csv()

instance.run_life_test(N=1)

***

## Future Work

Once the viability of the Mixing Station has been validated, we can look to use an API to call `experiment.run()` via the [Atinary software](https://scientia.atinary.com/sdlabs/academic/dashboard), enabling a self-driving lab setup where the constituent volumes are updated with each run.